1.Датасет ml-latest  
2.Вспомнить подходы, которые мы разбирали  
3.Выбрать понравившийся подход к гибридным системам  
4.Написать свою

придумывать особо нового ничего не буду просто поменяю алгоритм, чтобы он рекомендовал не фильм не из топ 10 близких по жанру, а найдём пользователя максимально похожего на него, и возьмём фильмы которые он посмотрел и уже исходя из этого выберем фильмы

поэтому все изменения я внёс только в результирующую функцию

In [31]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [40]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [42]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [43]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [44]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [45]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [46]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [47]:
%%time
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

Wall time: 5.23 s


In [48]:
algo

In [49]:
test_pred = algo.test(testset)

In [58]:
testset

[(482.0, 'Annie Hall (1977)', 3.0),
 (64.0, 'Backdraft (1991)', 3.0),
 (440.0, 'Close Encounters of the Third Kind (1977)', 4.5),
 (448.0, 'Annie (1982)', 3.0),
 (225.0, 'Psycho (1960)', 3.0),
 (510.0, 'Boogie Nights (1997)', 3.0),
 (6.0, 'Powder (1995)', 4.0),
 (590.0, 'Christmas Story, A (1983)', 4.0),
 (68.0, 'True Romance (1993)', 2.0),
 (339.0, 'The Founder (2016)', 4.5),
 (339.0, 'Quantum of Solace (2008)', 4.0),
 (436.0, 'With Honors (1994)', 3.0),
 (288.0, 'Davy Crockett, King of the Wild Frontier (1955)', 3.0),
 (307.0, '6th Man, The (Sixth Man, The) (1997)', 1.0),
 (217.0, 'Aristocats, The (1970)', 3.0),
 (23.0, 'Barry Lyndon (1975)', 2.0),
 (525.0, 'Rock, The (1996)', 4.0),
 (591.0, 'What Lies Beneath (2000)', 5.0),
 (572.0, 'Jude (1996)', 2.0),
 (452.0, 'Starship Troopers (1997)', 5.0),
 (187.0, 'Fountain, The (2006)', 3.5),
 (448.0, 'Uncle Buck (1989)', 3.0),
 (119.0, 'Million Dollar Baby (2004)', 5.0),
 (175.0, 'Footloose (1984)', 5.0),
 (474.0, 'Blue Velvet (1986)', 4.0)

In [50]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8657


0.865738988682464

In [51]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995)').est

2.5183559181571877

In [52]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [53]:
scores[:10]

[3.8228348328363193,
 3.5382812895393148,
 3.338454099188286,
 2.994008729591567,
 3.055000428546434,
 4.007370743160826,
 3.213172637272438,
 3.076467062462628,
 3.1881080233781587,
 3.4846501226032403]

In [54]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [55]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [56]:
movie_genres[0]

'Adventure Animation Children Comedy Fantasy'

In [22]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [65]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [67]:
predict

<1x20 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [25]:
movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
3576,4899,Black Knight (2001),Adventure|Comedy|Fantasy
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy
3582,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
3302,4467,"Adventures of Baron Munchausen, The (1988)",Adventure|Comedy|Fantasy


In [26]:
movies_with_ratings.sort_values('timestamp', inplace=True)

In [27]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

c:\users\sladk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [176]:
from sklearn.neighbors import NearestNeighbors
def recommend_for_user(user_id):
    current_user_id = user_id
    df_user = movies_with_ratings.pivot_table(index='userId', columns='title', values='rating', aggfunc='count')
    # сосздаём фичи для knn
    df_user.fillna(0, inplace=True)
    neigh = NearestNeighbors(n_neighbors=2)
    neigh.fit(df_user) 
    NearestNeighbors(n_neighbors=2) # берём топ 2 максимально похожим
    num = neigh.kneighbors(df_user[df_user.index==user_id])[1][0][1] # первым будет он сам, поэтому берём второго
    film = df_user[df_user.index==num].T     
    film = film[film[num]!=0]
    movies_to_score = list(film.reset_index()['title']) # и теперь оцениваем только его фильмы

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [177]:
recommend_for_user(4)

Wizard of Oz, The (1939) 4.133431802200936
Gladiator (1992) 4.0810045125112575
Amadeus (1984) 3.9212817831336206
Apollo 13 (1995) 3.91302996330351
Apocalypse Now (1979) 3.9045870726500365
Silence of the Lambs, The (1991) 3.8738439090200205
Persuasion (1995) 3.8654553818123514
Afterglow (1997) 3.8549911972540944
Fugitive, The (1993) 3.8406675202418006
Notting Hill (1999) 3.8390446169174894
